In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.optimizers.legacy import Adam

# Load the dataset
load_dir = './common-dataset'
loaded_dataset = tf.data.experimental.load(load_dir)

batch_size = 32
loaded_dataset = loaded_dataset.batch(batch_size).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

number_of_labels = 3862


Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 18236111630547889587


In [3]:
loaded_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 301, 1024), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3862), dtype=tf.int32, name=None))>

In [4]:
model = Sequential([
    SimpleRNN(64, return_sequences=True),
    SimpleRNN(64),
    Dense(128, activation='relu'),
    Dense(number_of_labels, activation='sigmoid')
])

# replace Adam optimizer with tf.keras.optimizers.legacy.Adam and learning rate 0.001


model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(loaded_dataset, epochs=10)


Epoch 1/10


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 18236111630547889587
2024-03-06 15:48:41.446771: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 138 of 1000


  1/158 [..............................] - ETA: 33:49 - loss: 0.6931 - accuracy: 0.1562

2024-03-06 15:48:43.083268: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.


158/158 [==============================] - 42s 182ms/step - loss: 0.6555 - accuracy: 0.1697
Epoch 2/10
158/158 [==============================] - 23s 89ms/step - loss: 0.5854 - accuracy: 0.1961
Epoch 3/10
158/158 [==============================] - 21s 90ms/step - loss: 0.5233 - accuracy: 0.1961
Epoch 4/10
158/158 [==============================] - 22s 94ms/step - loss: 0.4685 - accuracy: 0.1961
Epoch 5/10
158/158 [==============================] - 22s 93ms/step - loss: 0.4202 - accuracy: 0.1961
Epoch 6/10
158/158 [==============================] - 21s 92ms/step - loss: 0.3778 - accuracy: 0.1961
Epoch 7/10
158/158 [==============================] - 22s 92ms/step - loss: 0.3404 - accuracy: 0.1961
Epoch 8/10
158/158 [==============================] - 21s 90ms/step - loss: 0.3073 - accuracy: 0.1961
Epoch 9/10
158/158 [==============================] - 21s 90ms/step - loss: 0.2781 - accuracy: 0.1961
Epoch 10/10
158/158 [==============================] - 21s 89ms/step - loss: 0.2522 - accura

In [5]:
model